In [1]:
import pandas as pd
import csv
import multiprocessing as mp
from multiprocessing import Pool
import time

In [2]:
lnk='http://storage.googleapis.com/books/ngrams/books/20200217/eng/5-00009-of-19423.gz'

In [3]:
cur_file=[]
with open('/data/dharp/compounds/datasets/google/5-00009-of-19423','r') as f:
    cur_file=f.read().split('\n')
    cur_file=cur_file[:-1]

In [4]:
def list_processor(lst):
    all_rows={}
    if type(lst)==str:
        cur_fivegram=lst.split('\t')
        cur_counts=[]
        for i,line in enumerate(cur_fivegram):
            if i==0:
                key=line
            else:
                year,count=line.split(',')[:-1]
                cur_counts.append((year,count))

        cur_df=pd.DataFrame.from_records(cur_counts)
        cur_df.columns=['year','count']
        cur_df.set_index('year',inplace=True)

            #print(cur_row)
        all_rows[key]=cur_df
    else:   
        for cur_row in lst:
            #print(cur_row)
            cur_fivegram=cur_row.split('\t')
            cur_counts=[]
            #print(cur_fivegram)
            for i,line in enumerate(cur_fivegram):
                #print(line)
                if i==0:
                    key=line
                else:
                    year,count=line.split(',')[:-1]
                    cur_counts.append((year,count))

            cur_df=pd.DataFrame.from_records(cur_counts)
            cur_df.columns=['year','count']
            cur_df.set_index('year',inplace=True)

            all_rows[key]=cur_df
    combined_df=pd.concat(all_rows.values(), axis=0, keys=all_rows.keys())
    return combined_df

In [5]:
pool = Pool(250)

cur_time=time.time()
print("Starting parallelizing")

results=pool.map_async(list_processor,cur_file,chunksize=len(cur_file)//250)
pool.close()
pool.join()

df_list=results.get()

Starting parallelizing


In [6]:
new_df=pd.concat(df_list,sort=False)
new_df

count
                         year      
" Her _NOUN_ is Meredith 1976     1
                         1988     1
                         1991     1
                         1993     1
                         1998     1
...                             ...
" How _VERB_ your little 2015   171
                         2016   124
                         2017   157
                         2018   105
                         2019   105

[165642649 rows x 1 columns]

In [14]:
unique_df=new_df.index.unique(level=0).to_frame(index=False)
unique_df

,0
0,""" Her _NOUN_ is Meredith"
1,""" Heroes _ADV_ _._ """
2,""" However , can _PRON_"
3,""" Heliodorus _._ "" _ADP_"
4,""" He was a looker"
...,...
3296296,""" Hegel and the State"
3296297,""" He showed _ADV_ _._"
3296298,""" I 'd hope _ADV_"
3296299,""" Heading _PRT_ _ADV_ _ADV_"


In [39]:
unique_df.loc[unique_df[0].str.contains('_[A-Z]+_')]

,0
0,""" Her _NOUN_ is Meredith"
1,""" Heroes _ADV_ _._ """
2,""" However , can _PRON_"
3,""" Heliodorus _._ "" _ADP_"
5,""" I _VERB_ _DET_ lingo"
...,...
3296294,""" Hesitation _._ _._ _CONJ_"
3296297,""" He showed _ADV_ _._"
3296298,""" I 'd hope _ADV_"
3296299,""" Heading _PRT_ _ADV_ _ADV_"


In [33]:
ttt=unique_df.loc[~unique_df[0].str.contains('_.*_')]

In [36]:
ttt.loc[ttt[0].str.contains('VERB')]

,0
